In [1]:
import pandas as pd
import os
import numpyro
from refit_fvs.data.load import fia_for_diameter_growth_modeling
from refit_fvs.models.fit import fit_wykoff, fit_simpler_wykoff

In [2]:
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.75"
numpyro.set_platform("gpu")
numpyro.set_host_device_count(3)

# Select species
Identify species codes and the FVS variant from which bark ratio coefficients should be drawn.

In [3]:
FVS_ALPHA = "RC"
BARK_VAR = "WC"
FIA_SPCD = 242

In [4]:
BARK_COEFS = "../../data/raw/fvs_barkratio_coefs.csv"
bark = pd.read_csv(BARK_COEFS).set_index(["FIA_SPCD", "FVS_VARIANT"])

bark_b0, bark_b1, bark_b2 = bark.loc[FIA_SPCD, BARK_VAR][["BARK_B0", "BARK_B1", "BARK_B2"]]
bark_b0, bark_b1, bark_b2

(0.0, 0.94967, 1.0)

# Model-fitting parameters
These don't usually need to be updated when fitting a new species unless you want to add more chains or samples per chain.

In [5]:
NUM_CYCLES = 2
LOC_RANDOM = True
PLOT_RANDOM = True
CHECKPOINT_DIR = f"../../models/maicf/{FVS_ALPHA}/"
NUM_WARMUP = 1000
NUM_SAMPLES = 1000
NUM_CHAINS = 1
CHAIN_METHOD = "parallel"
NUM_BATCHES = 1
SEED = 42
PROGRESS_BAR = True
OVERWRITE = False

In [6]:
data, factors = fia_for_diameter_growth_modeling(
    path="../../data/interim/FIA_remeasured_trees_for_training.csv",
    filter_spp=[FIA_SPCD]
)
obs_variants, obs_locations, obs_plots = factors
data.describe()

,LOCATION,PLOT_ID,FIA_SPCD,MAICF,SLOPE,ASPECT,ELEV,DBH,CR,CR_NEXT,...,INC5YR,INC10YR,MEASYEAR,MEASMON,MEASYEAR_NEXT,MEASMON_NEXT,MEAS_INTERVAL,VAR_IDX,LOC_IDX,PLOT_IDX
count,3159.000000,3.159000e+03,3159.0,3159.000000,3159.000000,3159.000000,3159.000000,3159.000000,3159.000000,3159.000000,...,144.000000,144.000000,3159.000000,3159.000000,3159.000000,3159.000000,3159.0,3159.000000,3159.000000,3159.000000
mean,594.308642,4.920153e+11,242.0,124.536348,0.355169,2.718703,23.752770,19.209148,0.530133,0.517085,...,0.327431,0.681944,2005.581830,7.682178,2015.581830,7.615385,10.0,1.849953,7.558405,370.011396
std,120.557471,5.674997e+10,0.0,47.008278,0.274416,2.057025,11.929361,16.173999,0.201528,0.197920,...,0.223828,0.481642,2.413061,1.756099,2.413061,1.821142,0.0,0.968614,5.247243,238.235366
min,113.000000,6.010157e+10,242.0,10.000000,0.000000,0.000000,1.000000,5.000000,0.020000,0.010000,...,0.050000,0.100000,2001.000000,1.000000,2011.000000,3.000000,10.0,0.000000,0.000000,0.000000
25%,605.000000,4.101044e+11,242.0,92.410000,0.110000,0.698132,15.000000,7.300000,0.400000,0.375000,...,0.187500,0.350000,2004.000000,6.000000,2014.000000,6.000000,10.0,1.000000,3.000000,143.000000
50%,609.000000,5.305074e+11,242.0,121.020000,0.300000,2.652900,25.000000,12.300000,0.500000,0.500000,...,0.250000,0.550000,2006.000000,8.000000,2016.000000,8.000000,10.0,2.000000,6.000000,362.000000
75%,618.000000,5.307060e+11,242.0,151.000000,0.550000,4.625123,33.000000,28.500000,0.700000,0.650000,...,0.400000,0.900000,2008.000000,9.000000,2018.000000,9.000000,10.0,2.000000,10.000000,576.500000
max,800.000000,5.309066e+11,242.0,286.584000,1.200000,6.283185,51.000000,108.500000,0.990000,0.950000,...,1.200000,2.750000,2009.000000,12.000000,2019.000000,11.000000,10.0,5.000000,18.000000,805.000000


In [7]:
for tree_comp in ['bal', 'relht', 'ballndbh']:
    for stand_comp in ['ba', 'lnba', 'ccf']:
        for pooling in ['unpooled', 'partial']:
            model_name = f"wykoff_{pooling}_mixed_{tree_comp}-{stand_comp}"
            fit_wykoff(
                bark_b0,
                bark_b1,
                bark_b2,
                tree_comp,
                stand_comp,
                pooling,
                LOC_RANDOM,
                PLOT_RANDOM,
                NUM_CYCLES,
                data,
                model_name,
                CHECKPOINT_DIR,
                NUM_WARMUP,
                NUM_SAMPLES,
                NUM_CHAINS,
                CHAIN_METHOD,
                NUM_BATCHES,
                SEED,
                PROGRESS_BAR,
                OVERWRITE
            )

wykoff_unpooled_mixed_bal-ba
wykoff_unpooled_mixed_bal-ba_batch00


sample: 100%|█| 2000/2000 [14:15<00:00,  2.34it/s, 255 steps of size 2.25e-02. a


Done.
Samples saved at ../../models/maicf/RC//wykoff_unpooled_mixed_bal-ba_batch00.pkl
MCMC saved at ../../models/maicf/RC//wykoff_unpooled_mixed_bal-ba_batch00_mcmc.pkl
wykoff_partial_mixed_bal-ba
wykoff_partial_mixed_bal-ba_batch00


sample: 100%|█| 2000/2000 [24:46<00:00,  1.35it/s, 255 steps of size 2.05e-02. a


Done.
Samples saved at ../../models/maicf/RC//wykoff_partial_mixed_bal-ba_batch00.pkl
MCMC saved at ../../models/maicf/RC//wykoff_partial_mixed_bal-ba_batch00_mcmc.pkl
wykoff_unpooled_mixed_bal-lnba
wykoff_unpooled_mixed_bal-lnba_batch00


...

Done.
Samples saved at ../../models/maicf/RC//wykoff_partial_mixed_ballndbh-lnba_batch00.pkl
MCMC saved at ../../models/maicf/RC//wykoff_partial_mixed_ballndbh-lnba_batch00_mcmc.pkl
wykoff_unpooled_mixed_ballndbh-ccf
wykoff_unpooled_mixed_ballndbh-ccf_batch00


sample: 100%|█| 2000/2000 [14:20<00:00,  2.32it/s, 255 steps of size 2.33e-02. a


Done.
Samples saved at ../../models/maicf/RC//wykoff_unpooled_mixed_ballndbh-ccf_batch00.pkl
MCMC saved at ../../models/maicf/RC//wykoff_unpooled_mixed_ballndbh-ccf_batch00_mcmc.pkl
wykoff_partial_mixed_ballndbh-ccf
wykoff_partial_mixed_ballndbh-ccf_batch00


sample: 100%|█| 2000/2000 [24:35<00:00,  1.36it/s, 255 steps of size 2.47e-02. a


Done.
Samples saved at ../../models/maicf/RC//wykoff_partial_mixed_ballndbh-ccf_batch00.pkl
MCMC saved at ../../models/maicf/RC//wykoff_partial_mixed_ballndbh-ccf_batch00_mcmc.pkl


In [8]:
for pooling in ['unpooled', 'partial']:
    for tree_comp in ['bal', 'relht', 'ballndbh']:
        for stand_comp in ['ba', 'lnba', 'ccf']:
            model_name = f"simplerwykoff_{pooling}_mixed_{tree_comp}-{stand_comp}"
            fit_simpler_wykoff(
                bark_b0,
                bark_b1,
                bark_b2,
                tree_comp,
                stand_comp,
                pooling,
                LOC_RANDOM,
                PLOT_RANDOM,
                NUM_CYCLES,
                data,
                model_name,
                CHECKPOINT_DIR,
                NUM_WARMUP,
                NUM_SAMPLES,
                NUM_CHAINS,
                CHAIN_METHOD,
                NUM_BATCHES,
                SEED,
                PROGRESS_BAR,
                OVERWRITE
            )

simplerwykoff_unpooled_mixed_bal-ba
simplerwykoff_unpooled_mixed_bal-ba_batch00


sample: 100%|█| 2000/2000 [11:20<00:00,  2.94it/s, 255 steps of size 2.10e-02. a


Done.
Samples saved at ../../models/maicf/RC//simplerwykoff_unpooled_mixed_bal-ba_batch00.pkl
MCMC saved at ../../models/maicf/RC//simplerwykoff_unpooled_mixed_bal-ba_batch00_mcmc.pkl
simplerwykoff_unpooled_mixed_bal-lnba
simplerwykoff_unpooled_mixed_bal-lnba_batch00


sample: 100%|█| 2000/2000 [11:35<00:00,  2.88it/s, 255 steps of size 2.04e-02. a


Done.
Samples saved at ../../models/maicf/RC//simplerwykoff_unpooled_mixed_bal-lnba_batch00.pkl
MCMC saved at ../../models/maicf/RC//simplerwykoff_unpooled_mixed_bal-lnba_batch00_mcmc.pkl
simplerwykoff_unpooled_mixed_bal-ccf
simplerwykoff_unpooled_mixed_bal-ccf_batch00


...

Done.
Samples saved at ../../models/maicf/RC//simplerwykoff_partial_mixed_ballndbh-ba_batch00.pkl
MCMC saved at ../../models/maicf/RC//simplerwykoff_partial_mixed_ballndbh-ba_batch00_mcmc.pkl
simplerwykoff_partial_mixed_ballndbh-lnba
simplerwykoff_partial_mixed_ballndbh-lnba_batch00


sample: 100%|█| 2000/2000 [18:10<00:00,  1.83it/s, 255 steps of size 2.15e-02. a


Done.
Samples saved at ../../models/maicf/RC//simplerwykoff_partial_mixed_ballndbh-lnba_batch00.pkl
MCMC saved at ../../models/maicf/RC//simplerwykoff_partial_mixed_ballndbh-lnba_batch00_mcmc.pkl
simplerwykoff_partial_mixed_ballndbh-ccf
simplerwykoff_partial_mixed_ballndbh-ccf_batch00


sample: 100%|█| 2000/2000 [17:36<00:00,  1.89it/s, 255 steps of size 2.33e-02. a


Done.
Samples saved at ../../models/maicf/RC//simplerwykoff_partial_mixed_ballndbh-ccf_batch00.pkl
MCMC saved at ../../models/maicf/RC//simplerwykoff_partial_mixed_ballndbh-ccf_batch00_mcmc.pkl
